# Semantic Style GAN のモーフィング生成

## 1. 事前学習済みモデルをダウンロードする
学習済みモデルをpklファイルをダウンロードして，`pretrained/` に配置する．**DL : https://github.com/seasonSH/SemanticStyleGAN/releases/download/1.0.0/CelebAMask-HQ-512x512.pt**

## 2. 画像のサイズを事前学習済みモデルに合わせる
CelebAMask-HQでは512x512サイズであるので，入力画像もそのサイズにリサイズする．`images/`を作成して，入力画像を配置して以下のプログラムを動かす．元画像が上書きされるので注意が必要．

In [ ]:
!python resize512.py --indir images

## 3. 画像をモデルにinvertする（学習済みモデルに則して潜在変数を探索してファインチューニングをする．）
`images/`にある顔画像を全通りinvertする．従って，ある顔画像でinvertして，それでファインチューニングしたモデルを用いて別の顔画像で更にファインチューニングをする．顔画像は順不同なので${}_n C_2$個のファインチューニング済みモデルが作成される．潜在変数は`results/inversion/latent/*-*/`，ファインチューニング済みモデルは`/pretrained`に出力される．やり直すときは`/pretrained`のptファイルを削除しておくこと．

### 3.1. 学習済みモデルを用いて全ての顔画像に対してinvertする
作成されたファインチューニング済みモデルは`/results/inversion/weights`に保存される．

In [ ]:
!PYTHONPATH=.:$PYTHONPATH python visualize/invert.py --ckpt pretrained/CelebAMask-HQ-512x512.pt --imgdir images --outdir results/inversion --size 512 --finetune_step 300

### (3.2.) 潜在変数を確認する
潜在変数が本当に指定した顔に近づいているか，顔表情，マスク，スペクトルをを確認する．

In [ ]:
!python components_display.py

### 3.3. ファインチューニングした全てのモデルを用いて全ての顔画像に対してinvertする

In [ ]:
!python pair_inverting.py

## 4. モーフィングを作成する
`/pretrained`のモデル順にモーフィングを作成する．逆順のモーフィングは作成されないのが逆再生で対応できる．顔パーツ区間に分けながら潜在変数で線形モーフィングする．線形モーフィングは $ I^M=\alpha I^S+(1.0-\alpha) I^T \quad(0 \leq \alpha \leq 1.0)$ ではあるが，$ I^S $，$ I^T $は単に画素値ではなく潜在変数であるこに留意する．

In [ ]:
!python morphing.py